# Term Frequency - Inverse Document Frequency Vectorizer

Z.W.Miller - Copyright 2018

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy 
%matplotlib inline
plt.style.use('seaborn')

In [2]:
import sklearn
import matplotlib
import sys
libraries = (('Matplotlib', matplotlib), ('Numpy', np), ('Pandas', pd), ('Scipy', scipy), ('Sklearn', sklearn))

print("Python Version:", sys.version, '\n')
for lib in libraries:
    print('{0} Version: {1}'.format(lib[0], lib[1].__version__))


Python Version: 3.7.15 (default, Oct 12 2022, 19:14:55) 
[GCC 7.5.0] 

Matplotlib Version: 3.2.2
Numpy Version: 1.21.6
Pandas Version: 1.3.5
Scipy Version: 1.7.3
Sklearn Version: 1.0.2


In [3]:
import numpy as np
from collections import Counter
from string import punctuation

class tfidf_vectorizer:
    
    def __init__(self, max_features=None, ngrams = (1,1), tokenizer=None, remove_stopwords=False):
        """
        Term frequency, inverse document frequency vectorizer 
        reads the text provided, tokenizes it with the provided 
        tokenizer (or the default), then generates ngrams keeping 
        track of all ngrams as the vocabulary. Then it takes provided 
        texts and converts them into vectors by counting the 
        appearance of each ngram and tracking that for every document. 
        The counts are then scaled by the max term frequency and the
        inverse document frequency (see converter method). This new
        result is better than counts at picking out how important
        words are based on both usage and uniqueness. 
        ---
        KWargs:
        max_features: how many ngrams to allow in the vector, using the
        most common features first. If None, defaults to using all
        ngrams (int)
        ngrams: how many tokens to combine to form features. First element
        of tuple is starting point, second is ending point.
        tokenizer: what function to use to create tokens (must return 
        list of tokens)
        remove_stopwords: whether to include very common english words that
        do not add much value due to their commonness.
        """
        self.max_features = max_features
        self.vocabulary = {}
        self.ngrams = ngrams
        if tokenizer == None:
            self.tokenizer = self.tokenize
        else:
            self.tokenizer = tokenizer
        self.remove_stopwords = remove_stopwords
        self.stopwords = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 
                          'there', 'about', 'once', 'during', 'out', 'very', 'having', 
                          'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 
                          'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 
                          'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
                          'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 
                          'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 
                          'himself', 'this', 'down', 'should', 'our', 'their', 'while', 
                          'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 
                          'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 
                          'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 
                          'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 
                          'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 
                          'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 
                          'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 
                          'was', 'here', 'than'}
        
    def token_generator(self, X):
        """
        Generator that returns joined tokens as a single
        string to act as a feature. It generates the tokens
        by iterating through the allowed ngrams and combining
        the appropriate number of tokens into a string.
        """
        for i in range(self.ngrams[0],self.ngrams[1]+1):
            for ix, _ in enumerate(X):
                if ix+i < len(X)+1:
                    yield ' '.join(X[ix:ix+i])
    
    def tokenize(self, X):
        """
        Simple tokenizer that removes punctuation,
        lowercases the text, and breaks on spaces.
        Also removes stopwords and numeric values
        from being treated as words.
        """
        for symbol in punctuation:
            X = X.replace(symbol,'')
        final_token_list = [] 
        for token in X.lower().split():
            if self.remove_stopwords:
                if not self.check_stopwords(token):
                    try:
                        int(token)
                        float(token)
                    except:
                        final_token_list.append(token)  
            else:
                final_token_list.append(token)
        return final_token_list
        
    def check_stopwords(self, token):
        """
        Checks if the token is in our list of common
        stopwords, and returns a boolean.
         """
        return token in self.stopwords
    
    def fit(self, X):
        """
        Go through all provided training documents and
        create the list of vocabulary for known documents
        by looking at all ngrams and tracking how often
        those ngrams appear. If max_features is defined,
        only keep the most common tokens. Afterward,
        generate a token_to_id mapper and an id_to_token
        mapper.
        """
        for document in X:
            tokens = self.tokenizer(document)
            for token in self.token_generator(tokens):
                if token in self.vocabulary.keys():
                    self.vocabulary[token] += 1
                else:
                    self.vocabulary[token] = 1
        
        if self.max_features != None:
            temp_vocab = {}
            for key, value in Counter(self.vocabulary).most_common(self.max_features):
                temp_vocab[key] = value
            self.vocabulary = temp_vocab
            del temp_vocab
            
        self.token_to_id = {ky: ix for ix, ky in enumerate(sorted(self.vocabulary.keys()))}
        self.id_to_token = {ix: ky for ix, ky in enumerate(sorted(self.vocabulary.keys()))}
        
        
    def transform(self, X):
        """
        Go through all provided documents and use the known
        vocabulary to track how often each ngram appears in
        the document. At the end, stack all of the generated
        document vectors together. Convert them to tf-idf
        and skip the initial vector that's all 0's, which 
        is just there to act as a template.
        """
        vectorized_docs = np.zeros(len(self.vocabulary.keys()))
        for document in X:
            tokens = self.tokenizer(document)
            vectorized_doc = np.zeros(len(self.vocabulary.keys()))
            for token in self.token_generator(tokens):
                if token in self.vocabulary:
                    word_id = self.token_to_id[token]
                    vectorized_doc[word_id] += 1
            vectorized_docs = np.vstack((vectorized_docs,vectorized_doc))
        return self.convert_counts_to_tf_idf(vectorized_docs)[1:]
    def convert_counts_to_tf_idf(self, docs):
        """
        To convert from counts to TF-IDF, we first scale
        each value by the maximum in it's own column. This 
        lowers dependence on document length. Then we calculate
        log(number of documents/(1+documents containing this ngram)).
        This is the inverse document frequency (the one is to make
        combat division by 0). Each value is scaled as:
        term_frequency*inverse_document_frequency.
        """
        number_of_columns = docs.shape[1]
        number_of_docs = docs.shape[0]
        frequency_scalers = np.ones(number_of_columns)
        idf_terms = np.ones(number_of_columns)
        for col in range(number_of_columns):
            column_vals = docs.T[col]
            frequency_scalers[col] = np.max(column_vals)
            number_of_docs_containing = np.sum((column_vals > 0).astype(int))
            idf_terms[col] = np.log(number_of_docs/(1+number_of_docs_containing))
        docs = docs/frequency_scalers
        docs = docs*idf_terms
        
        return docs    

    def fit_transform(self, X):
        """
        Fit on X and then transform X and return it as vectors.
        """
        self.fit(X)
        return self.transform(X)
                               
    

In [4]:
cv = tfidf_vectorizer(ngrams=(1,3), max_features=None)
data = ['bob went to the store','dana, did not go to the store', 'the dog ran quickly toward the stoplight']
cv.fit(data)

In [5]:
cv.vocabulary

{'bob': 1,
 'went': 1,
 'to': 2,
 'the': 4,
 'store': 2,
 'bob went': 1,
 'went to': 1,
 'to the': 2,
 'the store': 2,
 'bob went to': 1,
 'went to the': 1,
 'to the store': 2,
 'dana': 1,
 'did': 1,
 'not': 1,
 'go': 1,
 'dana did': 1,
 'did not': 1,
 'not go': 1,
 'go to': 1,
 'dana did not': 1,
 'did not go': 1,
 'not go to': 1,
 'go to the': 1,
 'dog': 1,
 'ran': 1,
 'quickly': 1,
 'toward': 1,
 'stoplight': 1,
 'the dog': 1,
 'dog ran': 1,
 'ran quickly': 1,
 'quickly toward': 1,
 'toward the': 1,
 'the stoplight': 1,
 'the dog ran': 1,
 'dog ran quickly': 1,
 'ran quickly toward': 1,
 'quickly toward the': 1,
 'toward the stoplight': 1}

In [6]:
cv.transform(data)

array([[0.69314718, 0.69314718, 0.69314718, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.28768207, 0.        , 0.        , 0.        , 0.        ,
        0.28768207, 0.28768207, 0.28768207, 0.28768207, 0.        ,
        0.        , 0.        , 0.69314718, 0.69314718, 0.69314718],
       [0.        , 0.        , 0.        , 0.69314718, 0.69314718,
        0.69314718, 0.69314718, 0.69314718, 0.69314718, 0.        ,
        0.        , 0.        , 0.69314718, 0.69314718, 0.69314718,
        0.69314718, 0.69314718, 0.69314718, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.28768207, 0.        , 0.        , 0.        , 0.        ,
        0.28768207, 0.28768207, 0.28768207, 0.2

# Now let's turn on stopwords

In [7]:
cv = tfidf_vectorizer(ngrams=(1,3), max_features=None, remove_stopwords=True)
data = ['bob went to the store','dana, did not go to the store','the dog ran quickly toward the stoplight']
cv.fit(data)

In [8]:
cv.transform(data)

array([[0.69314718, 0.69314718, 0.69314718, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.28768207, 0.        ,
        0.        , 0.69314718, 0.69314718],
       [0.        , 0.        , 0.        , 0.69314718, 0.69314718,
        0.69314718, 0.        , 0.        , 0.        , 0.69314718,
        0.69314718, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.28768207, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.69314718, 0.69314718, 0.69314718, 0.        ,
        0.        , 0.69314718, 0.69314718, 0.69314718, 0.69314718,
        0.69314718, 0.69314718, 0.69314718, 0.        , 0.69314718,
        0.69314718, 0.        , 0.        ]])

# Now a larger dataset

In [9]:
from sklearn import datasets

categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball']
ng_train = datasets.fetch_20newsgroups(subset='train', 
                                       categories=categories, 
                                       remove=('headers', 
                                               'footers', 'quotes'))
data = ng_train.data
targets = ng_train.target

In [10]:
cv = tfidf_vectorizer(ngrams=(1,3), max_features=100, remove_stopwords=True)
cv.fit(data)

In [11]:
pd.DataFrame(cv.transform(data), columns=cv.token_to_id.keys())


,3d,also,another,anyone,argument,atheists,available,back,believe,best,...,using,version,want,way,well,without,work,would,year,years
0,0.0,0.060062,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.156879,0.0
1,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.338281,...,0.000000,0.0,0.0,0.0,0.107740,0.0,0.000000,0.156603,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.184121,0.0,0.0,0.0,0.000000,0.0,0.273365,0.000000,0.000000,0.0
1657,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.184121,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1658,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.215479,0.0,0.000000,0.156603,0.000000,0.0
1659,0.0,0.000000,0.0,0.0,0.11267,0.049139,0.0,0.0,0.101623,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.156603,0.000000,0.0


We can see this a pretty sparse set of vectors, so if we wanted to store this in a smaller format for transferring, we could convert it to a sparse matrix

In [12]:
vectors = cv.transform(data)

In [13]:
from scipy import sparse
sparse_matrix = sparse.csr_matrix(vectors)

In [14]:
print(sparse_matrix)

  (0, 1)	0.06006227187458042
  (0, 30)	0.06765414491261239
  (0, 44)	0.2501101112768755
  (0, 50)	0.19098441633434454
  (0, 53)	0.20920851641474703
  (0, 98)	0.15687879615922845
  (1, 58)	0.03735065491569401
  (2, 32)	0.10077767550087349
  (2, 44)	0.2501101112768755
  (2, 80)	0.7409400415083943
  (3, 32)	0.050388837750436746
  (3, 55)	0.1455548016565918
  (3, 73)	0.2510502196976964
  (4, 9)	0.33828084676288245
  (4, 12)	0.23529843583598417
  (4, 28)	0.13921538689209098
  (4, 45)	0.3016996232914419
  (4, 59)	0.07616333023847875
  (4, 79)	1.6003751191852191
  (4, 84)	0.13301613174287383
  (4, 94)	0.10773963463799348
  (4, 97)	0.15660264620151787
  (5, 3)	0.6770939708754349
  (5, 8)	0.20324637217713937
  (5, 12)	0.23529843583598417
  :	:
  (1658, 70)	0.09381389084551936
  (1658, 71)	0.2158281603387612
  (1658, 94)	0.21547926927598696
  (1658, 97)	0.15660264620151787
  (1659, 4)	0.11266962652717288
  (1659, 5)	0.04913884354693559
  (1659, 8)	0.10162318608856968
  (1659, 46)	0.1227456957740

Now let's use the output in a model

In [15]:
import sys
sys.path.append('../../zwml/naive_bayes/')

In [23]:
pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=4eaaa4fc16573d4e3aeec5fd8a918d5efbe95acaca23587f87c62d5b623446c5
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [25]:
pip install -U numpy scipy scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.naive_bayes import MultinominalNB 


In [34]:
(vectors, targets)

(array([[0.        , 0.06006227, 0.        , ..., 0.        , 0.1568788 ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.15660265, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.15660265, 0.        ,
         0.        ],
        [0.        , 0.06006227, 0.        , ..., 0.07830132, 0.        ,
         0.        ]]), array([2, 2, 0, ..., 0, 0, 1]))

In [33]:
(vectors, targets)

(array([[0.        , 0.06006227, 0.        , ..., 0.        , 0.1568788 ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.15660265, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.15660265, 0.        ,
         0.        ],
        [0.        , 0.06006227, 0.        , ..., 0.07830132, 0.        ,
         0.        ]]), array([2, 2, 0, ..., 0, 0, 1]))

In [30]:
from sklearn.dummy import DummyClassifier

dc = DummyClassifier()
dc.fit(vectors,targets)
dc.score(vectors,targets)

0.35942203491872365